# ∂Lux Tutorial - Basic Usage
---

The goal of this tutorial is to give you a base level understanding of ∂Lux, the objects you will interact with the most as you use it, and how the package as structured. It is strongly recommended that you follow this tutorial first before moving on to the next ones as it will cover most of the important gotchas that are likely to be encountered while using it! 

By the end of this tutorial you should know how to create, interact with and use ∂Lux models to simulate optical system


### Overview: 
- *Section 1*: What is ∂Lux
- *Section 2*: How does it work
- *Section 3*: Creating and interacting with ∂Lux objects
- *Section 4*: Simulating optical systems

---
---
# Section 1: What is ∂Lux?



## History

∂Lux is a full from-scratch rewrite of the ideas [morphine](https://github.com/benjaminpope/morphine), which is a fork of the popular optical simulation package '[poppy](https://github.com/mperrin/poppy)' using the autodiff library [Google Jax](https://github.com/google/jax) to do _derivatives_. We have built it from the ground up in [equinox](https://github.com/patrick-kidger/equinox), a powerful object-oriented library in Jax, to best take advantage of new features and permit easy development and integration with neural networks.

## The Basics

The goal of ∂Lux is to facilitate a paradaigm shift in the way in which optical modelling is approached. The mathematical and structural symmetry betweeen neural networks and optical systems allows for optical modelling to done with Machine Learning libraries that can take advatange of the massively powerful tools driving the modern ML revolution. By constructing differentiable optical models that harness the power of automatic differention in the ∂Lux framework allows entirely novel ways of approach conventianlly difficult problems!

For the uninitiated, automatic differentaion (auto-diff) is the mathematical tool that underpins the revolution in machine learning. The power of auto-diff ultimately lies in its ability to divorce the time it takes to optimise a model from the number of parameters being optimised in that model. This represents a *fundamental paradigm shift* in the way in which problems can be approached. Much time and effort has been focused in the past on making problems in optical modelling computationally tracatable, forcing compromises on what is learnt. This is no longer the case, directly optimising physics-based forwards models with millions of parameters is not only possible, but practical without requiring vast computation power. 

We have built ∂Lux using Jax - googles numpy-like auto-diff library and Equinox. Together these two packages allow us to build an optical simulator that takes full advantage of the bleeding edge of computer science. For example each individual PSF calcualtion is natively performed in parallel across however many computational resources are available without any work from the end-user. Similarly these models can be compiled at run time into XLA without. 

TBC...

∂Lux is a optical simulation framework that is designed to allow the construction of fully-differentiable optical models that can be optimised using automatic differention (auto-diff). Why does that matter? Auto-diff is the mathematics that allows for extremely complex models with billions of parameters to be optimised efficiently - its what allows the machine to 'learn'. By construcing differentible optical models we can massively increase their complexity and train these models in the same way, with the same tools as a neural network. This represents a paradigm shift in the way in which we can approach a diverse range of optical problems from everything from signal recovery to optical architecture. A large focus of this work has been on flexibility. 



∂Lux is built primarily from [Jax](https://jax.readthedocs.io/en/latest/) & [Equinox](https://docs.kidger.site/equinox/). Jax is googles numpy-like auto-diff library and Equinox is a new package that allows python classes to be registered in Jax as a PyTree, a data type that it works with natively. This allows us to write highly flexible object-oriented optical models with a numpy-like API that is natively differentiable. Furthrmore we can take advantage of industry leading tools such as [Optax](https://optax.readthedocs.io/en/latest/) the optimisation library used and maintained by the Google DeepMind team. 

Primary Advantages:
 - Numpy like API
 - Natively differenitable
 - Object-Oriented models
 - Vmap
 - Gpu
 - Jit
 - Gradients
 - Modern optimisation algorithms


---

## Package Overview

∂Lux has been built to be as simple and easy as possible for end-users, without abstracting them away from the underlying control-flow of the computations. The goal of this package is NOT to be a collection of optical modelling algorithms, but to be a FRAMEWORK that allows for easy construction of natively differentiably optical models in a robust way. With this goal we have focused heavily on flexibility, readability and constructing code to be natively differentiable.

Its structure is designed around the idea that optical systems operate in a mathematically analgous way to neural networks. 
<!--  A Neural network maps some input perform a seires of linear transformations and array opterations on some input vector through a series of layers. and optical systems transform some input wavefront using a series of optical surfaces and mirrors.  -->
In the same way that neural networks are constructed from layers, we represent our optical train as a linear list of operations performed on some input wavefront, also called layers. 

∂Lux functions using three classes:
 1. Layers
 2. `Wavefront()`
 3. `Optical System()`
    
The `OpticalSystem` class holds a list of Layers which define the operations performed on the input `Wavefront()` via the optical train, and passes the `Wavefront` object between those layers. We have already built a series of generic Layers that should allow users to simulate most optical systems, contained in two scirpts:
 1. `src/layers.py`
 2. `src/propagators.py`

With `layers.py` containing the classes to perform transformation operations on the wavefront (modify phase, amplitude, interpolate) and `propagators.py` containing the classes used to propagate that wavefront through the optical train.

- 

As open open source software we also encourage users to [build thier own custom Layer classes](tutorial_link_goes_here.com) and integrate them into ∂Lux!






There are two main types of classes that form the foundation of ∂Lux, the `OpticalSystem()` and the layers. In order to construct a model of an optical system one simply defines the series of operations/transforms that is performed on the input wavefront in a list, which is passed as an argument to the `OpticalSystem()` class. Each transformation or operation is a single 'layer' in that list. For a very simple optical a typical list of layers would look something like this:

```
layers = [
  CreateWavefront(wf_npix, wf_size),
  TiltWavefront(),
  CircualrAperture(wf_npix),
  NormaliseWavefront(),
  MFT(det_npix, fl, det_pixsize)
]
```

This list of layers can then be turned into an optical system -> `OpticalSystem(layers)`. We now have a fully differentiable optical model!

The `OpticalSystem()` is the main class that we will interact with and does most of the heavy lifting, so lets a take a detailed look at what this class does.

---

# The `OpticalSystem()` object!

The OpticalSystem object is the primary object of dLux, so here is a quick overview.

> dLux curently does not check that inputs are correctly shaped/formatted in order to making things work appropriately (under development)

## Inputs:


### layers: list, required
 - A list of layers that defines the tranformaitons and operations of the system (typically optical)
 
### wavels: ndarray, optional
 - An array of wavelengths in meters to simulate
 - The shape must be 1d - stellar spectrums are controlled through the weights parameter
 - No default value is set if not provided and this will throw an error if you try to call functions that depend on this parameter
 - It is left as optional so that functions that allow wavelength input can be called on objects without having to pre-input wavelengths
 
### positions: ndarray, optional
 - An array of (x,y) stellar positions in units of radians, measured as deivation of the optical axis. 
 - Its input shape should be (Nstars, 2), defining an x, y position for each star. 
 - If not provided, the value defaults to (0, 0) - on axis

### fluxes: ndarray, optional
 - An array of stellar fluxes, its length must match the positions inputs size to work properly
 - Theoretically this has arbitrary units, but we think of it as photons
 - Defaults to 1 (ie, returning a unitary flux psf if not specified)

### weights: ndarray, optional
 - An array of stellar spectral weights (arb units)
 - This can take multiple shapes
     - Default is to weight all wavelengths equally (top-hat)
     - If a 1d array is provided this is applied to all stars, shape (Nwavels)
     - if a 2d array is provided each is applied to each individual star, shape (Nstars, Nwavels)
 - Note the inputs values are always normalised and will not directly change total output flux (inderectly it can change it by weighting more flux to wavelengths with more aperture losses, for example)

### dithers: ndarray, optional
 - An arary of (x, y) positional dithers in units of radians
 - Its input shape should be (Nims, 2), defining the (x,y) dither for each image
 - if not provided, defualts to no-dither

### detector_layers: list, optional
 - A second list of layer objects designed to allow processing of psfs, rather than wavefronts
 - It is applied to each image after psfs have been approraitely weighted and summed
     
     
## Functions:

### __call__()
> Primary call function applying all parameters of the scene object through the systems
 - Takes no inputs, returning a image, or array of images
 - The primary function designed to apply all of the inputs of the class in order to generate the appropriate output images
 - Automatically maps the psf calcualtion over both wavelength and input position for highly efficient calculations
 - It takes no inputs as to allow for easier coherent optimsation of the whole system 
 
### propagate_mono(wavel):
> Propagates a single monochromatic wavelength through only the layers list
 - Inputs:
     - wavel (float): The wavelength in meters to be modelled through the system
     - offset (ndarray, optional): the (x,y) offest from the optical axis in radians
 - Returns: A sigle monochromatic PSF
 
### propagate_single(wavels)
> Propagataes a single broadband stellar source through the layers list
 - Inputs:
     - wavels (ndarray): The wavelengths in meters to be modelled through the system
     - offset (ndarray, optional): the (x,y) offest from the optical axis in radians
     - weights (ndarray, optional): the realative weights of each wavelength, 
         - No normalisation is applied to the weights to allow user flexibility
         - Unitary weights will output a total sum of 1
 - Returns: A single broadband PSF
 
 
### debug_prop(wavels)
> Propagataes a single wavelength through while storing the intermediary value of the wavefront and pixelscale between each operation. This is designed to help build and debug unexpected behaviour. It is functionally a mirror of propagate_mono() that stored intermediary values/arrays
 - Inputs:
     - wavels (ndarray): The wavelengths in meters to be modelled through the system
     - offset (ndarray, optional): the (x,y) offest from the optical axis in radians
 - Returns: [A single monochromatic PSF, intermediate wavefront, intermediate pixelscales]
     
     


